In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade "ibm-watson>=5.3.0"
!pip install unidecode
!pip install word2number

In [ ]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.websocket import RecognizeCallback, AudioSource
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import unidecode
import string

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def speechToText(file_path, speech_to_text):
  # open the audio file and run recognition
  audio_file = open(file_path, 'rb')
  content_type = 'audio/' + file_path.split('.')[1]
  response = speech_to_text.recognize(model = 'en-US_BroadbandModel', # english model
                                      audio = audio_file, 
                                      content_type = content_type,
                                      smart_formatting = True, 
                                      profanity_filter = False,
                                      end_of_phrase_silence_time = 0.6
                                     ).get_result()
  audio_file.close()
  return response

def createRecognizer(api_info):
  # setup
  authenticator = IAMAuthenticator(api_info['apikey'])
  speech_to_text = SpeechToTextV1(authenticator=authenticator)
  speech_to_text.set_service_url(api_info['url'])

  return speech_to_text

In [ ]:
# sample audio file for test purposes, replace this with actual audio input
file_idx = 1
folder_path = '/content/drive/MyDrive/colab data/speech_data/'
sample_list = ['sample1.flac', 'sample2.mp3', 'sample3.flac']

file_path = folder_path + sample_list[file_idx]

# api definition
api_info = {
            "apikey": "pWBFUaj4ktgQ2rXojtRVVUdWvyoFD1HphLVaqOfHDkiB",
            "iam_apikey_description": "Auto-generated for key 5e021eae-f934-40bd-b370-e54abc1656d6",
            "iam_apikey_name": "Auto-generated service credentials",
            "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
            "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/15ac184c657344819e27057364c1dcb3::serviceid:ServiceId-9e5f761a-bace-4fb0-8465-b2ef8ae046c3",
            "url": "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/2d429bd2-a6dc-4b3d-bb5c-c795d82d400d"
           }

# set up the speech recognizer
speech_to_text = createRecognizer(api_info)

# run speech recognition 
response = speechToText(file_path, speech_to_text)

In [ ]:
# process the result
result = []
for sentence in response['results']:
  text = sentence['alternatives'][0]['transcript'].lower().strip()

  text = text.translate(str.maketrans('', '', string.punctuation))

  text = re.sub("\\n", " ", text)
  #remove user_id and IP address
  text = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", '', text)
  #remove text staring with user
  text = re.sub("\[\[User.*",'', text)
  #remove url and links
  text = re.sub('^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$', ' ', text)

  result.append(text)

# examine the result
for sent in result:
  print(sent)

thank you for choosing the olympus dictation management system
the olympus dictation management system gives you the power to manage your dictation transcriptions and documents seamlessly and to improve the productivity of your daily work
for example you can automatically sends the dictation files or transcribed documents to your assistant of the otha via email or ftp
if youre using the speech recognition software the speech recognition engine works in the background to support your document creation
we hope you enjoy the simple flexible reliable and secure solutions from olympus


In [ ]:
def remove_stopword(s):
    s = str(s)
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    #final_sentence = ''
    final_sentence = ' '.join(str(x) for x in filtered_sentence)
    return final_sentence
        
def lemm(s):
    s = str(s)
    word_list = nltk.word_tokenize(s)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

In [ ]:
df = pd.DataFrame(result, columns=['sentence'])

df['sentence'] = df['sentence'].map(remove_stopword)
df['sentence'] = df['sentence'].map(lemm)
df['sentence'] = df['sentence'].map(strip_html_tags)
df['sentence'] = df['sentence'].map(remove_accented_chars)
df['sentence'] = df['sentence'].apply(lambda x: x.lower())

df.head()

,sentence
0,thank choosing olympus dictation management sy...
1,olympus dictation management system give power...
2,example automatically sends dictation file tra...
3,youre using speech recognition software speech...
4,hope enjoy simple flexible reliable secure sol...
